In [1]:
import os
import json

import pandas as pd

In [2]:
DATA_PATH = '../data/crops'

In [7]:
color_class_mapper = {class_name: class_index for class_name, class_index in zip(os.listdir(DATA_PATH), range(4))}
with open('color_classes.json', 'w') as jf:
    json.dump(color_class_mapper, jf)

In [11]:
image_paths = []
pet_color = []
pet_attrs = []
for subfolder in os.scandir(DATA_PATH):
    for tail_length_folder in os.scandir(subfolder.path):
        for image_file in os.scandir(tail_length_folder.path):
            image_paths.append(image_file.path)
            pet_color.append(color_class_mapper[subfolder.name])
            tail_length = 1 if tail_length_folder.name == 'long' else 0
            pet_attrs.append(tail_length)

In [15]:
df = pd.DataFrame.from_dict({
    'image': image_paths,
    'color': pet_color,
    'long_tail': pet_attrs
})

In [20]:
from sklearn.model_selection import train_test_split

In [27]:
X_train, X_test = train_test_split(df, test_size=0.3, random_state=39, shuffle=True, stratify=df['color'])

In [29]:
X_test, X_val = train_test_split(X_test, test_size=0.5, random_state=39, shuffle=True, stratify=X_test['color'])

In [30]:
df.loc[X_test.index, 'subset'] = 'test'
df.loc[X_train.index, 'subset'] = 'train'
df.loc[X_val.index, 'subset'] = 'val'

In [32]:
df.to_csv('../data/train.csv', index=False)

In [1]:
import sys

sys.path.insert(0, '.')

In [2]:
import pandas as pd

In [3]:
from dataset import AnimalDataset

In [5]:
df = pd.read_csv('../data/train.csv')
dset = AnimalDataset(df)

In [6]:
for i in dset:
    image, color, tail = i
    print(image.shape)
    print(color, tail)
    break

(41, 58, 3)
0 1


In [2]:
from utils.config import get_config

In [3]:
config = get_config('config.yml')

In [4]:
config

Config(training=TrainingConfig(seed=39, epochs=30, device='cuda:0', num_workers=4, batch_size=32, ema=0.8), path=PathConfig(csv='folds_crops.csv', weights_folder='./outputs'), model=ModelConfig(name='tf_efficientnet_b0_ns', pretrained=True, file_name='color_and_tail_model.pth', color_dropout_rate=0.5, color_num_features=1024, tail_num_features=1024, tail_dropout_rate=0.5), optimizer=OptimizerConfig(algo='torch.optim.Adam', lr=0.001, weight_decay=5e-05), scheduler=SchedulerConfig(algo='torch.optim.lr_scheduler.StepLR', step_size=6, gamma=0.5), criterion=CriterionConfig(algo='torch.nn.MSELoss'), augmentations=AugmentationsConfig(train={'transform': {'__class_fullname__': 'albumentations.core.composition.Compose', 'bbox_params': None, 'keypoint_params': None, 'p': 1, 'transforms': [{'__class_fullname__': 'albumentations.augmentations.transforms.Resize', 'always_apply': True, 'height': 120, 'width': 120, 'p': 1.0}, {'__class_fullname__': 'albumentations.augmentations.transforms.HorizontalF

In [5]:
from model import AnimalModel

In [6]:
model = AnimalModel(config)

1024


In [7]:
model

AnimalModel(
  (backbone): EfficientNet(
    (conv_stem): Conv2dSame(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (act1): SiLU(inplace=True)
    (blocks): Sequential(
      (0): Sequential(
        (0): DepthwiseSeparableConv(
          (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (bn1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
          (act1): SiLU(inplace=True)
          (se): SqueezeExcite(
            (conv_reduce): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (act1): SiLU(inplace=True)
            (conv_expand): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (gate): Sigmoid()
          )
          (conv_pw): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn2): BatchNorm2d(16, eps=0.001, momentum=0.1, affine=True, track_r

In [8]:
import torch

tensor = torch.randn(1, 3, 224, 224)

model(tensor)

(tensor([[ 0.0189,  0.0244, -0.0604, -0.0619]], grad_fn=<AddmmBackward>),
 tensor([[0.0067]], grad_fn=<AddmmBackward>))